# Face-Swap GAN (FSGAN) Implementation Guide

## 1. Install Required Libraries

In [1]:
!pip install dlib



  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dlib: filename=dlib-19.24.6-cp39-cp39-win_amd64.whl size=3069725 sha256=f90000113d4d84fd288c4535328e0eba3c5f93a437ced026cd534902276dfe79
  Stored in directory: c:\users\rishu\appdata\local\pip\cache\wheels\f5\ba\ea\ad13d4a963e5dbbbaf4dce5fef93353e2e7586ba8b7787221c
Successfully built dlib


In [ ]:
!pip install tensorflow opencv-python dlib numpy matplotlib tqdm --user


- **TensorFlow**: For building the GAN model.
- **OpenCV**: For image processing tasks.
- **dlib**: For facial landmark detection.
- **NumPy**: For numerical computations.
- **Matplotlib**: For visualizing results.
- **tqdm**: For progress tracking during training.


## 2. Import Libraries

In [2]:

# Importing necessary libraries
import tensorflow as tf  # TensorFlow for creating deep learning models
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, UpSampling2D, Concatenate, Dense, Flatten  # Essential Keras layers
from tensorflow.keras.models import Model  # To define and compile models
import cv2  # OpenCV for image processing and face alignment
import dlib  # For facial landmark detection
import numpy as np  # NumPy for numerical operations
import matplotlib.pyplot as plt  # Matplotlib for visualizing data
from tqdm import tqdm  # Tqdm for showing progress bars



- TensorFlow and Keras: For constructing and training the GAN.
- OpenCV (`cv2`) and dlib: For face detection and preprocessing.
- NumPy: To handle image arrays.
- Matplotlib: To visualize image outputs.
- tqdm: Displays progress bars during loops.


## 3. Load and Preprocess the Data

In [4]:
# Initializing dlib's face detector and shape predictor
detector = dlib.get_frontal_face_detector()  # Detects frontal faces in an image
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Predicts 68 landmarks for face alignment

# Function to preprocess images
def preprocess_image(image_path, img_size=128):
    """
   
    """
    img = cv2.imread(image_path)  # Read the image from the file path
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR (OpenCV format) to RGB
    
    faces = detector(img)  # Detect faces in the image
    if len(faces) == 0:  # If no faces are detected, raise an error
        raise ValueError("No face detected!")
    
    for face in faces:  # Iterate over detected faces (assumes one face per image)
        landmarks = predictor(img, face)  # Predict facial landmarks
        aligned_face = align_face(img, landmarks)  # Align face based on detected landmarks (function to define)
        resized_face = cv2.resize(aligned_face, (img_size, img_size))  # Resize the aligned face to the desired size
        return resized_face / 255.0  # Normalize pixel values to [0, 1] for GAN input



- **detector**: Detects faces in the input images.
- **predictor**: Identifies 68 facial landmarks, essential for alignment.
- **align_face**: A helper function to crop and align faces based on landmarks (to be defined later).
- **Normalization**: Converts pixel values to the range [0, 1] for better GAN performance.


## 4. Define the FSGAN Architecture

In [11]:
def build_generator(input_shape=(128, 128, 3)):
    
    inputs = Input(shape=input_shape)  # Input layer with specified shape
    
    # Downsampling layers
    x = Conv2D(64, kernel_size=4, strides=2, padding="same")(inputs)  # Convolution with stride 2 to downsample
    x = LeakyReLU(alpha=0.2)(x)  # LeakyReLU for better gradient flow during training
    
    # Bottleneck (adds more convolution layers for feature extraction)
    for _ in range(4):  # Repeat 4 times to increase feature extraction depth
        x = Conv2D(128, kernel_size=4, strides=2, padding="same")(x)
        x = LeakyReLU(alpha=0.2)(x)
    
    # Upsampling layers (reconstruct the image from bottleneck features)
    for _ in range(4):  # Repeat 4 times to scale up the spatial dimensions
        x = UpSampling2D(size=2)(x)  # Upsample by a factor of 2
        x = Conv2D(128, kernel_size=4, padding="same")(x)  # Convolution after upsampling
        x = LeakyReLU(alpha=0.2)(x)
    
    outputs = Conv2D(3, kernel_size=7, activation="tanh", padding="same")(x)  # Output layer with tanh activation
    return Model(inputs, outputs, name="Generator")  # Return the complete generator model


- **Input**: Takes in a 128x128x3 image.
- **Downsampling**: Extracts features while reducing spatial dimensions.
- **Bottleneck**: Encodes image features into a compact representation.
- **Upsampling**: Reconstructs the output image to the original resolution.


In [12]:
def build_discriminator(input_shape=(128, 128, 3)):
    
    inputs = Input(shape=input_shape)  # Input layer with specified shape
    
    # Downsampling layers
    x = Conv2D(64, kernel_size=4, strides=2, padding="same")(inputs)  # Convolution with stride 2 to downsample
    x = LeakyReLU(alpha=0.2)(x)  # LeakyReLU activation for gradient stability
    
    for _ in range(3):  # Add additional convolutional layers
        x = Conv2D(128, kernel_size=4, strides=2, padding="same")(x)  # Downsampling further
        x = LeakyReLU(alpha=0.2)(x)  # Activation function for non-linearity
    
    # Flattening and output layer
    x = Flatten()(x)  # Flatten the features into a single vector
    x = Dense(1, activation="sigmoid")(x)  # Output a single probability (real or fake)
    
    return Model(inputs, x, name="Discriminator")  # Return the complete discriminator model



- **Conv2D Layers**: Extract features at multiple levels.
- **LeakyReLU**: Handles negative activations, aiding in stable GAN training.
- **Dense Layer**: Produces a single output indicating real or fake.


## 5. Compile the GAN

In [13]:

def build_fsgan(generator, discriminator):
    discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002), loss="binary_crossentropy", metrics=["accuracy"])
    discriminator.trainable = False
    
    inputs = Input(shape=(128, 128, 3))
    generated_image = generator(inputs)
    valid = discriminator(generated_image)
    
    gan_model = Model(inputs, valid)
    gan_model.compile(optimizer=tf.keras.optimizers.Adam(0.0002), loss="binary_crossentropy")
    return gan_model



- **Generator**: Learns to create realistic face-swapped images.
- **Discriminator**: Differentiates real from fake images.
- **GAN Model**: Combines both networks for joint training.


# 6. Train the FSGAN


In [14]:
def train_fsgan(generator, discriminator, gan_model, epochs, batch_size, dataset):
    for epoch in range(epochs):
        for i in tqdm(range(len(dataset) // batch_size)):
            real_images = dataset[i * batch_size:(i + 1) * batch_size]
            noise = np.random.normal(0, 1, (batch_size, 128, 128, 3))
            fake_images = generator.predict(noise)
            
            # Train discriminator
            real_labels = np.ones((batch_size, 1))
            fake_labels = np.zeros((batch_size, 1))
            d_loss_real = discriminator.train_on_batch(real_images, real_labels)
            d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
            
            # Train generator
            g_loss = gan_model.train_on_batch(noise, np.ones((batch_size, 1)))
        
        print(f"Epoch {epoch+1}/{epochs} | D Loss: {d_loss_real[0]:.4f}, G Loss: {g_loss:.4f}")



- **Real and Fake Labels**: Used for supervised training of the discriminator.
- **GAN Loss**: Encourages the generator to create realistic images.

# 7. Test and Visualize Results



In [15]:
def test_fsgan(generator, input_image):
    generated_image = generator.predict(np.expand_dims(input_image, axis=0))[0]
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(input_image)
    plt.subplot(1, 2, 2)
    plt.title("Face-Swapped Image")
    plt.imshow((generated_image + 1) / 2)  # De-normalize
    plt.show()


In [ ]:
# Example usage
generator = build_generator()



ValueError: Input 0 of layer "Discriminator" is incompatible with the layer: expected shape=(None, 128, 128, 3), found shape=(None, 64, 64, 3)

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load the pre-trained FSGAN model (assuming you've trained or downloaded it)
model = load_model('path_to_your_trained_model.h5')

# Load your images (make sure to have two face images for swapping)
image1 = cv2.imread('path_to_image1.jpg')
image2 = cv2.imread('path_to_image2.jpg')

# Pre-process the images (resize and normalize as required by the model)
image1_resized = cv2.resize(image1, (256, 256))  # Adjust to your model's input size
image2_resized = cv2.resize(image2, (256, 256))

image1_normalized = image1_resized / 255.0
image2_normalized = image2_resized / 255.0

# Concatenate the images (or adjust according to your model input)
input_images = np.concatenate([image1_normalized, image2_normalized], axis=0)
input_images = np.expand_dims(input_images, axis=0)

# Generate the swapped face
output = model.predict(input_images)

# Post-process the output if needed (for example, denormalize the images)
output_image = output[0] * 255.0  # Assuming the model outputs in [0, 1] range
output_image = output_image.astype(np.uint8)

# Display the results
plt.imshow(output_image)
plt.axis('off')
plt.show()
